In [1]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras

In [2]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft,
    	sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    	sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [3]:
def parse_audio_files(files_dir):
    features, labels = np.empty((0,193)), np.empty(0)
    for dirpath, dirnames, filenames in os.walk(files_dir):
        for filename in filenames:
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(filename)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            features = np.vstack([features,ext_features])
            if (filename.split('_')[2]=="Positivo"):
                labels = np.append(labels, 1)
            if (filename.split('_')[2]=="Negativo"):
                labels = np.append(labels, 0)
            #labels = np.append(labels, int(filename.split('_')[2]))
    return np.array(features, dtype=np.float32), np.array(labels, dtype = np.int8)

In [8]:
parent_dir = "D:/ITT/ServicioSocial/DatosTosCompartir/Train/"
save_dir = "D:/ITT/ServicioSocial/DatosTosCompartir/processed/"
sub_dirs = np.array(['positivo','negativo'])
for sub_dir in sub_dirs:
    features, labels = parse_audio_files(parent_dir)
    np.savez("{0}{1}".format(save_dir, sub_dir), features=features,
             labels=labels)

C:\Users\andre\anaconda3\envs\android\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=919
  n_fft, y.shape[-1]
C:\Users\andre\anaconda3\envs\android\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=460
  n_fft, y.shape[-1]
C:\Users\andre\anaconda3\envs\android\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=230
  n_fft, y.shape[-1]
C:\Users\andre\anaconda3\envs\android\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=115
  n_fft, y.shape[-1]
C:\Users\andre\anaconda3\envs\android\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=831
  n_fft, y.shape[-1]
C:\Users\andre\anaconda3\envs\android\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small fo

In [9]:
def get_network():
    input_shape = (193,)
    num_classes = 2
    keras.backend.clear_session()

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(256, activation="relu", input_shape=input_shape))
    model.add(keras.layers.Dense(128, activation="relu", input_shape=input_shape))
    model.add(keras.layers.Dense(64, activation="relu", input_shape=input_shape))
    model.add(keras.layers.Dense(num_classes, activation = "softmax"))
    model.compile(optimizer=keras.optimizers.Adam(1e-4),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"])

    return model

In [10]:
accuracies = []
folds = np.array(['positivo','negativo'])
load_dir = "D:/ITT/ServicioSocial/DatosTosCompartir/processed/"
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(folds):
    x_train, y_train = [], []
    for ind in train_index:
        data = np.load("{0}/{1}.npz".format(load_dir,folds[ind]))
        x_train.append(data["features"])
        y_train.append(data["labels"])
    x_train = np.concatenate(x_train, axis = 0)
    y_train = np.concatenate(y_train, axis = 0)

    data = np.load("{0}/{1}.npz".format(load_dir,folds[test_index][0]))
    x_test = data["features"]
    y_test = data["labels"]
    

    model = get_network()
    model.fit(x_train, y_train, epochs = 50, batch_size = 24, verbose = 0)
    l, a = model.evaluate(x_test, y_test, verbose = 0)
    accuracies.append(a)
    print("Loss: {0} | Accuracy: {1}".format(l, a))

print("Accuracy: {0}".format(np.mean(accuracies)))

Loss: 0.08805905774668957 | Accuracy: 0.9783637523651123
Loss: 0.08413838804337405 | Accuracy: 0.9803921580314636
Accuracy: 0.9793779850006104


In [7]:
os.chdir("D:/ITT/ServicioSocial/DatosTosCompartir/Train/")